In [36]:
# For Python2.6 compatibility
from __future__ import print_function

import pretty_midi
import numpy as np
# For plotting
import matplotlib.pyplot as plt
%matplotlib inline
# For putting audio in the notebook
import IPython.display
import copy

inputtedMidi = pretty_midi.PrettyMIDI('eminor.mid')
times, tempo_changes = inputtedMidi.get_tempo_changes()

pitchInventory = [0 for i in range(12)]
for instrument in inputtedMidi.instruments:
    for note in instrument.notes:
        pitchInventory[note.pitch%12] += 1

copiedPitchCount = copy.deepcopy(pitchInventory)
copiedPitchCount.sort()
copiedPitchCount.reverse()
cardinality = len(copiedPitchCount)
for i in range(len(copiedPitchCount)):
    if copiedPitchCount[i] == 0:
        cardinality = i
        break
threshold = copiedPitchCount[6]
for i in range(7, len(copiedPitchCount)):
    if copiedPitchCount[i] < threshold:
        threshold = copiedPitchCount[i]
        break

        
adjustedPitchInventory = [(i if i > threshold else 0) for i in pitchInventory]
majorSteps = [2, 2, 1, 2, 2, 2]
harmonicMinorSteps = [2, 1, 2, 2, 1, 3]
keyType = -1
tonic = -1

In [37]:
for i in range(0, 12):
    if keyType != -1:
        break
    
    if adjustedPitchInventory[i] == 0:
        continue
    
    fits = True
    currentPitch = i
    for j in range(min(len(majorSteps), cardinality-1)):
        currentPitch = (currentPitch + majorSteps[j]) % 12
        if not adjustedPitchInventory[currentPitch]:
            fits = False
            break
    if fits:
        keyType = 0
        tonic = i
        break
    
    fits = True
    currentPitch = i
    for j in range(min(len(majorSteps), cardinality-1)):
        currentPitch = (currentPitch + harmonicMinorSteps[j]) % 12
        if not adjustedPitchInventory[currentPitch]:
            fits = False
            break
    if fits:
        keyType = 1
        tonic = i
        break

print(keyType)
print(tonic)

1
4


In [167]:
import pretty_midi
import numpy as np
from functools import cmp_to_key
import heapq
import bisect

def get_examples(name, tonic):
    input_midi = pretty_midi.PrettyMIDI(name)
    
    times, tempo_changes = input_midi.get_tempo_changes()
    print(60/tempo_changes[0])   
    
    midi_notes = input_midi.instruments[0].notes;
    
    beats = input_midi.get_beats()
    beat_ticks = [input_midi.time_to_tick(i) for i in beats]
    
    def compare(x, y):
        result = False
        if (abs(x.start - y.start) < 0.000001):
            result = -1 if (x.pitch > y.pitch) else 1
        else:
            result = -1 if (x.start < y.start) else 1
        return result
    midi_notes.sort(key=cmp_to_key(compare))
    
    start_end_events = []
    for i in range(len(midi_notes)):
        start_end_events.append((input_midi.time_to_tick(midi_notes[i].start), midi_notes[i].pitch, i+1))
        start_end_events.append((input_midi.time_to_tick(midi_notes[i].end), midi_notes[i].pitch, -(i+1)))
        
    def compareNoteEvents(x, y):
        result = 0
        # sort by pitch
        if x[0] == y[0]:
            result = -1 if (x[1] > y[1]) else 1
        else:
            result = -1 if (x[0] < y[0]) else 1
        return result
    
    start_end_events.sort(key=cmp_to_key(compareNoteEvents))
    
    highest_pitch_heap = []
    current_time = start_end_events[0][0]
    current_melody = start_end_events[0][1]
    current_notes = set()
    heapq.heappush(highest_pitch_heap, (-start_end_events[0][1], start_end_events[0][2]))
    current_notes.add(start_end_events[0][2])
    
    melody = []
    accompaniment = []
    
    for i in range(1, len(start_end_events)):
        current_event = start_end_events[i]
        
        if current_time != current_event[0]: # new time, append to melody and accompaniment
            # get rid of outdated
            
            while highest_pitch_heap and not highest_pitch_heap[0][1] in current_notes:
                heapq.heappop(highest_pitch_heap)
            
            if not highest_pitch_heap:
                if current_event[2] > 0:
                    current_notes.add(current_event[2])
                    heapq.heappush(highest_pitch_heap, (-current_event[1], current_event[2]))
                else:
                    current_notes.remove(-current_event[2])
                continue
            
            melody_id = highest_pitch_heap[0][1]
            melody.append((current_time, -highest_pitch_heap[0][0]))
            
            chord = []
            for i in current_notes:
                if i != melody_id:
                    chord.append(midi_notes[i-1].pitch)
            accompaniment.append((current_time, tuple(chord)))
            current_time = current_event[0]
        
        if current_event[2] > 0:
            current_notes.add(current_event[2])
            heapq.heappush(highest_pitch_heap, (-current_event[1], current_event[2]))
        else:
            current_notes.remove(-current_event[2])
    
#     print(melody)
#     print(accompaniment)

    melody_pitches = [[0 for i in range(12)] for j in range(len(beat_ticks))]
    accompaniment_pitches = [[0 for i in range(12)] for j in range(len(beat_ticks))]
    
    for note in melody:
        melody_pitches[bisect.bisect_right(beat_ticks, note[0])-1][(note[1] - tonic + 12) % 12] = 1
    
    for chord in accompaniment:
        for pitch in chord[1]:
            accompaniment_pitches[bisect.bisect_right(beat_ticks, chord[0])-1][(pitch - tonic + 12) % 12] = 1
    
    print(len(accompaniment))
    
    
#     test = [5, 9, 14, 15, 19]
#     print(bisect.bisect_right(test, 20))
    
#     notes_heap = []
#     current_time = midi_notes[0].start
#     for note in midi_notes:
#         current_time = note.start
#         heappush((-note.pitch, note.end))
        
#         highest = heappop(h)
#         while(highest[1].end < )
#         print((note.start, note.end, note.pitch))


    return None
# chpn-p15_format0.mid
get_examples('chpn-p15_format0.mid', 1)

0.8062349999999999
1213


# 